<a href="https://colab.research.google.com/github/Ziannnniiii/2025_AI_TI1B/blob/main/Praktikum%20Job5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Praktikum

# 1. Top 5 Customers yang memiliki nilai transaksi paling banyak

In [2]:
import pandas as pd

df_transactions = pd.read_csv('transactions_1k.csv')
df_customers = pd.read_csv('customers.csv')
df_gasstations = pd.read_csv('gasstations.csv')

df_transactions['Price'] = pd.to_numeric(df_transactions['Price'], errors='coerce')

customer_totals = df_transactions.groupby('CustomerID')['Price'].sum().reset_index(name='TotalPrice')

top_5_customers = pd.merge(customer_totals, df_customers, on='CustomerID', how='left')

top_5_details = pd.merge(top_5_customers, df_transactions, on='CustomerID', how='left')
top_5_details = pd.merge(top_5_details, df_gasstations, on='GasStationID', how='left')

top_5_details = top_5_details.drop_duplicates('CustomerID', keep='first')

result = top_5_details[[
    'CustomerID', 'TransactionID', 'Date', 'Time', 'CardID', 'GasStationID',
    'ProductID', 'Amount', 'Price',  'TotalPrice', 'Segment_x',
     'ChainID', 'Country', 'Segment_y'
]].head(5)

result = result.rename(columns={
    'Currency_x': 'Transaction_Currency',
    'Currency_y': 'Customer_Currency',
    'Segment_x': 'Customer_Segment',
    'Segment_y': 'Station_Segment'
})

result

,CustomerID,TransactionID,Date,Time,CardID,GasStationID,ProductID,Amount,Price,TotalPrice,Customer_Segment,ChainID,Country,Station_Segment
0,96,401,2012-08-24,09:26:00,764858,5300,322,29,28.17,81.61,LAM,77,SVK,Premium
4,272,414,2012-08-24,11:40:00,622621,5261,322,62,59.76,59.76,LAM,246,SVK,Noname
5,651,739,2012-08-25,19:12:00,595267,3121,2,31,771.85,771.85,KAM,4,CZE,Premium
6,679,579,2012-08-25,09:05:00,752560,267,2,107,92.26,92.26,LAM,35,CZE,Discount
7,1057,315,2012-08-24,13:02:00,585106,3748,23,1,123.91,969.07,SME,4,CZE,Premium


# 2. Top 5 Gas stations yang memiliki nilai transaksi paling banyak

In [3]:
import pandas as pd

df = pd.read_csv('transactions_1k.csv')

df_top5 = df.groupby('GasStationID')['Price'].sum().reset_index()

df_top5 = df_top5.sort_values(by='Price', ascending=False).head(5)

df_top5



,GasStationID,Price
395,4347,8557.94
0,48,6868.67
291,3250,6744.47
142,1155,6701.69
217,2417,6065.16


# 3. Top 5 Jenis produk yang memiliki nilai transaksi paling banyak

In [4]:
import pandas as pd

df_transactions = pd.read_csv('transactions_1k.csv')
df_products = pd.read_csv('products.csv')

df_merged = pd.merge(df_transactions, df_products, on='ProductID', how='left')

df_top5 = df_merged.groupby(['ProductID', 'Description'])['Price'].sum().reset_index()

df_top5 = df_top5.sort_values(by='Price', ascending=False).head(5)

df_top5 = df_top5.rename(columns={'Price': 'Total Transaction Value'})

df_top5

,ProductID,Description,Total Transaction Value
0,2,Nafta,345669.45
4,9,Diesel +,27261.03
1,5,Natural,24639.46
11,20,Nafta Plus,7233.72
3,8,Natural +,3999.92


# 4. deskripsi statistik untuk masing masing hari (23,24,25,26) pada dataset transaction_1k

In [5]:
import pandas as pd

df = pd.read_csv('transactions_1k.csv')

df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')

df['Day'] = df['Date'].dt.day

df_filtered = df[df['Day'].isin([23, 24, 25, 26])]

stats_amount = df_filtered.groupby(df_filtered['Date'].dt.strftime('%Y-%m-%d'))['Amount'].describe()
stats_sales = df_filtered.groupby(df_filtered['Date'].dt.strftime('%Y-%m-%d'))['Price'].describe()

stats_combined = pd.concat([stats_amount, stats_sales], axis=1, keys=['Amount', 'Sales'])

stats_combined


Amount                                                        \
            count       mean        std  min    25%   50%    75%    max   
Date                                                                      
2012-08-23   10.0  29.600000  24.900692  1.0  21.25  26.5  35.25   91.0   
2012-08-24  492.0  19.936992  21.949630  0.0   5.00  19.0  27.00  264.0   
2012-08-25  425.0  18.880000  20.376114  0.0   4.00  17.0  26.00  154.0   
2012-08-26   73.0  21.219178  18.932810  0.0   6.00  20.0  29.00  101.0   

            Sales                                                           \
            count        mean         std    min      25%     50%      75%   
Date                                                                         
2012-08-23   10.0  685.379000  534.046487  76.42  481.170  619.44  795.095   
2012-08-24  492.0  409.939736  488.269481   1.76   76.125  332.45  615.395   
2012-08-25  425.0  426.940071  453.134207   6.09  102.220  370.38  600.850   
2012-08-26   73.0  487.431370  429.192499   4.74  142.840  437.03  675.520   

                     
                max  
Date                 
2012-08-23  1994.71  
2012-08-24  5762.49  
2012-08-25  3600.84  
2012-08-26  2303.75

# 5. Carilah waktu terbaik (hari dan jam) dimana paling banyak user gas station melakukan transaksi !

In [6]:
import pandas as pd

df = pd.read_csv('transactions_1k.csv')

df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

df['Date'] = df['Datetime'].dt.strftime('%Y-%m-%d')
df['Hour'] = df['Datetime'].dt.hour

result = df.groupby(['Date', 'Hour']).agg({
    'Amount': 'sum',
    'Price': 'sum'
}).reset_index()

result


,Date,Hour,Amount,Price
0,2012-08-23,9,79,1785.68
1,2012-08-23,20,26,720.89
2,2012-08-23,21,91,1994.71
3,2012-08-23,22,69,1603.51
4,2012-08-23,23,31,749.00
...,...,...,...,...
56,2012-08-26,9,310,7146.06
57,2012-08-26,10,120,2829.68
58,2012-08-26,11,67,1501.13
59,2012-08-26,12,152,3428.88


# 6 Dari segi bisnis understanding, tujuan utama dari analisis data customer ini adalah
1. Tujuan pertama adalah untuk mengidentifikasi produk bahan bakar dari stasiun gas yang paling diminati pelanggan. Hal ini memungkinkan produsen untuk meningkatkan persediaan produk tersebut.
2. Tujuan kedua adalah untuk menentukan stasiun gas dengan performa penjualan tertinggi. Dengan demikian, produsen dapat mengalokasikan lebih banyak stok ke stasiun tersebut.
3. Tujuan terakhir adalah untuk memahami waktu-waktu puncak pembelian produk oleh pelanggan. Informasi ini berguna untuk mempersiapkan operasional penjualan secara optimal pada jam-jam tersebut.